In [20]:
# imports
import pretty_midi
import warnings

### Example of MIDI file with empty tracks

In [34]:
black_or_white = pretty_midi.PrettyMIDI('/Users/huongngo/Desktop/RESEARCH/text-to-midi/captioning/examples/Black or White copy.mid')

/Users/huongngo/opt/anaconda3/envs/gehirn/lib/python3.11/site-packages/pretty_midi/pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [48]:
for instrument in black_or_white.instruments:
    print(instrument.notes[:2])

[Note(start=18.873909, end=19.186408, pitch=71, velocity=110), Note(start=19.132529, end=19.471968, pitch=76, velocity=114)]
[Note(start=2.068964, end=2.074352, pitch=36, velocity=111), Note(start=6.460125, end=6.465513, pitch=36, velocity=89)]
[Note(start=2.068964, end=2.370688, pitch=28, velocity=116), Note(start=10.086200, end=10.355596, pitch=40, velocity=125)]
[Note(start=35.167000, end=37.235964, pitch=66, velocity=70), Note(start=35.167000, end=37.235964, pitch=71, velocity=70)]
[Note(start=2.613145, end=2.726291, pitch=69, velocity=116), Note(start=2.607757, end=2.737067, pitch=64, velocity=107)]
[Note(start=93.103380, end=93.146483, pitch=40, velocity=91), Note(start=93.189587, end=93.324285, pitch=40, velocity=95)]
[Note(start=46.842638, end=47.058155, pitch=40, velocity=95), Note(start=46.842638, end=47.074319, pitch=45, velocity=101)]
[Note(start=103.458976, end=103.593674, pitch=68, velocity=87), Note(start=103.458976, end=103.604450, pitch=79, velocity=87)]
[Note(start=48

In [37]:
black_or_white.key_signature_changes

[]

In [39]:
black_or_white.get_tempo_changes()

(array([0.]), array([116.00008507]))

A warning is returned stating that there is a tempo, key or time signature event on one of the tracks after the first one. Here, track is not referred to the MIDI file tracks, but the pretty_midi tracks. The first track always contains the metadata of the song (key signature, time signature etc.). It seems like there is an empty track when there is a tempo, key or time signature event on one of the tracks after the first one. 

The empty track can't be removed but we can flag and ignore that track. It can only be manually modified. 

### Example of same MIDI file that has been modified to not have empty tracks

In [36]:
black_or_white_m = pretty_midi.PrettyMIDI('/Users/huongngo/Desktop/RESEARCH/text-to-midi/captioning/examples/black_or_white_m.mid')

In [49]:
for instrument in black_or_white.instruments:
    print(instrument.notes[:2])

[Note(start=18.873909, end=19.186408, pitch=71, velocity=110), Note(start=19.132529, end=19.471968, pitch=76, velocity=114)]
[Note(start=2.068964, end=2.074352, pitch=36, velocity=111), Note(start=6.460125, end=6.465513, pitch=36, velocity=89)]
[Note(start=2.068964, end=2.370688, pitch=28, velocity=116), Note(start=10.086200, end=10.355596, pitch=40, velocity=125)]
[Note(start=35.167000, end=37.235964, pitch=66, velocity=70), Note(start=35.167000, end=37.235964, pitch=71, velocity=70)]
[Note(start=2.613145, end=2.726291, pitch=69, velocity=116), Note(start=2.607757, end=2.737067, pitch=64, velocity=107)]
[Note(start=93.103380, end=93.146483, pitch=40, velocity=91), Note(start=93.189587, end=93.324285, pitch=40, velocity=95)]
[Note(start=46.842638, end=47.058155, pitch=40, velocity=95), Note(start=46.842638, end=47.074319, pitch=45, velocity=101)]
[Note(start=103.458976, end=103.593674, pitch=68, velocity=87), Note(start=103.458976, end=103.604450, pitch=79, velocity=87)]
[Note(start=48

In [45]:
pretty_midi.key_number_to_key_name(black_or_white_m.key_signature_changes[0].key_number)

'C Major'

In [46]:
black_or_white_m.get_tempo_changes()

(array([0.]), array([116.00008507]))